In [1]:
from pyDWSIMopt.interface import simulation
import numpy as np
from scipy import optimize
from pprint import pprint

ModuleNotFoundError: No module named 'simulation'

In [ ]:
# Loading DWSIM simulation into Python (Simulation object)
sim_smr = Simulation(path2sim="C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep_MSHE_MITApy.dwxmz")
sim_smr.savepath = "C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep_MSHE_MITApy2.dwxmz"
sim_smr.Add_refs()
sim_smr.Connect()

In [ ]:
def fobj8n_2exp(sim_smr, x):
    mr1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("MR-1")
    comp1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-1") 
    comp2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-2") 
    comp3 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-3") 
    comp4 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COMP-4") 
    pump1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-01") 
    pump2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-02") 
    vlv1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("VALV-01")
    cool8 = sim_smr.flowsheet.GetFlowsheetSimulationObject("COOL-08")
    lng = sim_smr.flowsheet.GetFlowsheetSimulationObject("LNG-1")
    if sim_smr.x is None:
        sim_smr.x = np.zeros(len(x))
    if np.linalg.norm(sim_smr.x - np.asarray(x))>1e-10:
        sep1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("SEP-02")
        sep2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("SEP-03")
        sep1.GraphicObject.Activate = False
        sep2.GraphicObject.Activate = False

        mr1.SetOverallCompoundMassFlow(0,x[1])
        mr1.SetOverallCompoundMassFlow(1,x[2])
        mr1.SetOverallCompoundMassFlow(2,x[3])
        mr1.SetOverallCompoundMassFlow(5,x[4])
        mr1.SetOverallCompoundMassFlow(7,x[0])
        vlv1.OutletPressure = x[5]
        comp4.POut = x[6]
        cool8.OutletTemperature = x[7]

        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-03").Phases[1].Properties.massfraction < 1e-5:
            pump1.GraphicObject.Active = False
            # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("PUMP-01").GraphicObject.Active)
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
        else:
            pump1.GraphicObject.Active = True
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)     #avoid bug
            sim_smr.flowsheet.ConnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-29").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject,-1,-1)
        sep1.GraphicObject.Active = True
        sep1.Calculate()
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-05").Phases[1].Properties.massfraction < 1e-5:
            pump2.GraphicObject.Active = False
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
        else:
            pump2.GraphicObject.Active = True
            pump2.Calculate()
            sim_smr.flowsheet.DisconnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject)
            sim_smr.flowsheet.ConnectObjects(
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-30").GraphicObject,
                sim_smr.flowsheet.GetFlowsheetSimulationObject("MIX-02").GraphicObject,-1,-1)
        sep2.GraphicObject.Active = True
        sep2.Calculate()
        sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        # print(comp1.DeltaQ + comp2.DeltaQ + comp3.DeltaQ + comp4.DeltaQ)
        # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA1-Calc").OutputVariables['mita'])
        # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA2-Calc").OutputVariables['mita'])
        # time.sleep(0.05)
        # sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        # time.sleep(0.05)
        # print(comp1.DeltaQ + comp2.DeltaQ + comp3.DeltaQ + comp4.DeltaQ)
        # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA1-Calc").OutputVariables['mita'])
        # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA2-Calc").OutputVariables['mita'])
        # sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        # time.sleep(0.05)
        # print(comp1.DeltaQ + comp2.DeltaQ + comp3.DeltaQ + comp4.DeltaQ)
        # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA1-Calc").OutputVariables['mita'])
        # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA2-Calc").OutputVariables['mita'])
        # sim_smr.interface.CalculateFlowsheet2(sim_smr.flowsheet)
        # print(comp1.DeltaQ + comp2.DeltaQ + comp3.DeltaQ + comp4.DeltaQ)
        # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA1-Calc").OutputVariables['mita'])
        # print(sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA2-Calc").OutputVariables['mita'])
        sumW = (comp1.DeltaQ + comp2.DeltaQ + comp3.DeltaQ + comp4.DeltaQ)
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-03").Phases[1].Properties.massfraction > 1e-5:
            sumW += pump1.DeltaQ 
        if sim_smr.flowsheet.GetFlowsheetSimulationObject("MSTR-05").Phases[1].Properties.massfraction > 1e-5:    
            sumW += pump2.DeltaQ
        sumW = sumW/lng.GetMassFlow()/3600
        mita1 = sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA1-Calc").OutputVariables['mita']
        mita2 = sim_smr.flowsheet.GetFlowsheetSimulationObject("MITA2-Calc").OutputVariables['mita']
    else:
        sumW = sim_smr.f
        mita1 = sim_smr.g
        mita2 = sim_smr.g
    sim_smr.x = x
    sim_smr.f = sumW
    sim_smr.g = min(mita1, mita2)
    return sumW, min(mita1, mita2)

In [ ]:
print(fobj8n_2exp(sim_smr,x0))
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,sim_smr.savepath,True)

In [ ]:
# Initial simulation setup
x0 = np.array( [0.25/3600, 0.70/3600, 1.0/3600, 1.10/3600, 1.80/3600, 2.50e5, 50.00e5, -60+273.15] )
x_top = np.array( [0.252779/3600, 0.441063/3600, 1.411068/3600, 0.805952/3600, 1.851257/3600, 3.342832e5, 34.97221e5, -27.2958+273.15] )
sim_smr.fobj = lambda x: fobj8n_2exp(sim_smr, x)
sumW, mita = sim_smr.fobj(1.0*x0)
print(sim_smr.x,
sim_smr.f,
sim_smr.g)
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,sim_smr.savepath,True)

In [ ]:
args=(sim_smr)
xtol=0.01
ftol=0.01
maxiter=50 # +- 5 seconds per iteration
print("starting optimization")
iteration = 0
regularizer = np.array([1e4,1e4,1e4,1e4,1e4, 1e-5, 1e-6, 1e-2])
#rever \/
bounds_raw = np.array( [0.6*np.asarray(x0), 1.4*np.asarray(x0)] )
bounds_raw[0][-1] = 153
bounds_raw[1][-1] = 233
bounds_reg = regularizer*bounds_raw
print(bounds_reg)
bounds = optimize.Bounds(bounds_reg[0], bounds_reg[1])
g = lambda x: fobj8n_2exp(sim_smr,np.asarray(x)/regularizer)[1]
nonlinear_constraint = optimize.NonlinearConstraint(g, 3, np.inf, jac='2-point', hess=optimize.BFGS())
f = lambda x: fobj8n_2exp(sim_smr,np.asarray(x)/regularizer)[0]
# print(fobj8n_2exp(sim_smr,x_top))

In [ ]:
fobj8n_2exp(sim_smr, x_top)
pprint(vars(sim_smr))
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet, sim_smr.savepath,True)

In [ ]:
# Local optimization with trust-region -> working to some extent
result = optimize.minimize( f, np.asarray(x0)*regularizer, 
            method='trust-constr', jac='2-point', hess=optimize.BFGS(),
            constraints=[nonlinear_constraint], bounds=bounds, callback=None,
            options={'verbose': 3, 
                     'xtol': xtol, 
                     'maxiter': 0.1*maxiter, 
                     'finite_diff_rel_step': None, 
                     'initial_tr_radius': 0.1} )

In [ ]:
print(f(result.x))
print(g(result.x))
# sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet, sim_smr.savepath,True)
print(result.x)
# x_top
bounds

In [ ]:
# Global optimization with Differential Evolution
result = optimize.differential_evolution(f, bounds, constraints=[nonlinear_constraint], 
            maxiter=40, popsize=1, seed=420, disp=True, polish=False)

In [ ]:
print(f(result.x))
print(g(result.x))
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet, sim_smr.savepath,True)
print(result.x)
pprint(result)

In [ ]:
# Global optimization with PSO
from sko.PSO import PSO
fpen = lambda x: f(x)+1000*max(0,3-g(x))
result_pso = PSO(func=fpen, n_dim=8, pop=20, max_iter=50, lb=bounds_reg[0], ub=bounds_reg[1], verbose=True)
result_pso.record_mode = True
result_pso.run()

In [ ]:
import matplotlib.pyplot as plt

print(f(result_pso.gbest_x))
print(g(result_pso.gbest_x))
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet, sim_smr.savepath,True)
print(result_pso.gbest_x)
pprint(result_pso)
print(fpen(result_pso.gbest_x))

plt.plot(result_pso.gbest_y_hist)
plt.show()

In [ ]:
import jsons

json_result = jsons.dump(result_pso)
